In [3]:
# Importing the required libraries
import PyPDF2
import requests
from io import BytesIO
import string
import requests
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
